In [ ]:
import pandas as pd
from functools import reduce

#preparing cleaned dataset for analysis

#merging datas from second half of 2023
df1 = pd.read_csv('202312-divvy-tripdata.csv')
df2 = pd.read_csv('202311-divvy-tripdata.csv')
df3 = pd.read_csv('202310-divvy-tripdata.csv')
df4 = pd.read_csv('202309-divvy-tripdata.csv')
df5 = pd.read_csv('202308-divvy-tripdata.csv')
df6 = pd.read_csv('202307-divvy-tripdata.csv')
df7 = pd.read_csv('202306-divvy-tripdata.csv')

data_frames = [df1, df2, df3,df4, df5, df6, df7]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=["ride_id","rideable_type","started_at","ended_at",
                                                               "start_station_name","start_station_id","end_station_name",
                                                               "end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"],
                                            how='outer'), data_frames)

#obtaining datas without NULL values 
df=df_merged.dropna() 

#processing data for analysis

# 1. Time when started
df['t1'] = pd.to_datetime(df['started_at'])
df['t2'] = pd.to_datetime(df['ended_at'])
df['time'] = df['t1'].dt.strftime("%H:%M:%S")
df['hour'] = df['t1'].dt.strftime("%H")

# 2. Ride length in minutes 
df['ride_length'] = (df['t2'] - df['t1']).dt.total_seconds() / 60
df = df.loc[(df['ride_length']>1) & (df['ride_length']<1440)]
df['ride_length_minutes'] = df['ride_length'].apply(lambda x: '{:02}:{:02}'.format(*divmod(int(x), 60)))

# 3. Day of the week 
df['day_of_week'] = df['t1'].dt.dayofweek
df['day_of_week'] = df['day_of_week'].map({0: 'MON', 1: 'TUE', 2: 'WED', 3: 'THU', 
                                           4: 'FRI', 5: 'SAT', 6: 'SUN'})
# 4. Day  
df['day'] = pd.DatetimeIndex(df['t1']).day

# 5. Month
df['month'] = pd.DatetimeIndex(df['t1']).month

# 6. Year
df['year'] = pd.DatetimeIndex(df['t1']).year

df.head()
#analyzing the data

#Average, Minimum and Maximum of the length of ride
#Average
df2 = df.loc[df['member_casual'] == 'casual']
df3=df.loc[df['member_casual'] == 'member']
meanc=df2['ride_length'].mean()
meanm=df3['ride_length'].mean()
avg = pd.DataFrame ([meanc,meanm], index = ["casual","member"], columns = ["mean"])

#Minimum
df2 = df.loc[df['member_casual'] == 'casual']
df3=df.loc[df['member_casual'] == 'member']
meanc=df2['ride_length'].min()
meanm=df3['ride_length'].min()
avg = pd.DataFrame ([meanc,meanm], index = ["casual","member"], columns = ["min"])

#Maximum
df2 = df.loc[df['member_casual'] == 'casual']
df3=df.loc[df['member_casual'] == 'member']
meanc=df2['ride_length'].max()
meanm=df3['ride_length'].max()
avg = pd.DataFrame ([meanc,meanm], index = ["casual","member"], columns = ["max"])

#Greatest number of riders in which days in a week
#Casual riders
df = df.loc[df['member_casual'] == 'casual']
df['day_of_week'].value_counts()

#Member riders
df = df.loc[df['member_casual'] == 'member']
df['day_of_week'].value_counts()

#Popular start hours
#Casual riders
df = df.loc[df['member_casual'] == 'casual']
df['hour'].value_counts()

#Member riders
df = df.loc[df['member_casual'] == 'member']
df['hour'].value_counts()

#Popular start stations 
#Casual riders
df = df.loc[df['member_casual'] == 'casual']
df['start_station_name'].value_counts()

#Member riders
df = df.loc[df['member_casual'] == 'member']
df['start_station_name'].value_counts()

#Popular routes 
#Casual riders
df['routes'] = df['start_station_name'].astype(str) +"-"+ df["end_station_name"]
df = df.loc[df['member_casual'] == 'casual']
df['routes'].value_counts()

#Member riders
df['routes'] = df['start_station_name'].astype(str) +"-"+ df["end_station_name"]
df = df.loc[df['member_casual'] == 'member']
df['routes'].value_counts()